In [ ]:
#| default_exp logs.services.file

In [68]:
#| exporti
import os
import csv
import json
from typing import Optional,Literal

from dc_logger.client.base import ServiceHandler, ServiceConfig, LogEntry
from dc_logger.client.exceptions import LogConfigError, LogHandlerError,LogWriteError


In [69]:
#| exporti

from dc_logger.client.base import ServiceConfig, OutputMode
from dataclasses import dataclass

In [70]:
#| export

@dataclass(kw_only=True)
class File_ServiceConfig(ServiceConfig):
    """Configuration for file-based logging output"""
    destination: str
    output_mode: Literal["File"] = "File"
    format: Literal["json", "text", "csv"] = "text"
    append: bool = True  # optional flag for overwrite or append

    def validate_config(self) -> bool:
        if not self.destination:
            raise LogConfigError("File destination must be provided.")
        if self.format not in ("json", "text", "csv"):
            raise LogConfigError(f"Unsupported file format: {self.format}")
        return True


In [71]:
#| export

class FileHandler(ServiceHandler):
    """Handler for file output"""

    def __init__(self, service_config: File_ServiceConfig):
        super().__init__(service_config)
        service_config.validate_config()

        self.file_path = service_config.destination
        self.append_mode = "a" if service_config.append else "w"
        self._ensure_directory_exists()

    def _ensure_directory_exists(self):
        """ Create parent directories in case they don't exist."""
        file_dir = os.path.dirname(self.file_path)
        if not file_dir:
            return
        try:
            os.makedirs(file_dir,exist_ok = True)
        except PermissionError as e:
            raise LogHandlerError(f"Permissions denied creating directory {file_dir}: {e}")
        except OSError as e:
            raise LogHandlerError(f"OS error creating the directory {file_dir}: {e}")
            
    async def _write_json(self, entry: LogEntry) -> bool:
        try:
            with open(self.file_path, self.mode, encoding="utf-8") as f:
                json.dump(entry.to_dict(),f,ensure_ascii=False)
                f.write("\n")
            return True
        except Exception as e:
            raise LogWriteError(f"Error writing JSON to file {self.file_path}: {e}")

    async def _write_text(self, entry: LogEntry) -> bool:
        try:
            line = f"[{entry.timestamp}] {entry.level.value if hasattr(entry.level, 'value') else entry.level} " \
                   f"{entry.logger}: {entry.message}\n"
            with open(self.file_path, self.mode, encoding="utf-8") as f:
                f.write(line)
            return True
        except Exception as e:
            raise LogWriteError(f"Error writing text to file {self.file_path}: {e}")

    async def _write_csv(self, entry: LogEntry) -> bool:
        try:
            file_exists = os.path.exists(self.file_path)
            with open(self.file_path, self.mode, newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=["timestamp", "level", "logger", "message"])
                if not file_exists or os.path.getsize(self.file_path) == 0:
                    writer.writeheader()
                writer.writerow(entry.to_dict())
            return True
        except Exception as e:
            raise LogWriteError(f"Error writing CSV to file {self.file_path}: {e}")

    async def write(self, entry: LogEntry, output_type: Optional[str] = None) -> bool:
        """Write a single log entry to the file"""
        try:
            output_type = output_type or self.service_config.format

            if output_type == "json":
                return await self._write_json(entry)
            elif output_type == "csv":
                return await self._write_csv(entry)
            elif output_type == "text":
                return await self._write_text(entry)
            else:
                raise LogConfigError(f"Unsupported output type: {output_type}")

        except Exception as e:
            raise LogWriteError(f"Error writing to file {self.file_path}: {e}")

    async def flush(self) -> bool:
        """File writes are immediate; no flush needed"""
        return True


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('./base.ipynb')